Load and prepare the data 

In [16]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

advisorskpi=pd.read_csv("multi_employee_dataset.csv")

#Display the first few rows

print(advisorskpi.head())

        Week Employee_ID    AHT  Schedule_Adherence    ACW  RONA  \
0   1/1/2024      EMP001  16.26               93.08  0.140     2   
1   1/8/2024      EMP001  18.01               96.17  1.272     0   
2  1/15/2024      EMP001  17.78               86.93  1.476     0   
3  1/22/2024      EMP001  13.32               98.28  1.497     2   
4  1/29/2024      EMP001   7.93               85.78  1.311     0   

   Evaluation_Completed  Efficiency  Compliance  Logging  Professionalism  \
0                  True        85.0        95.0     95.0             95.0   
1                  True        95.0        95.0     90.0            100.0   
2                 False         NaN         NaN      NaN              NaN   
3                 False         NaN         NaN      NaN              NaN   
4                 False         NaN         NaN      NaN              NaN   

   Exceptions  Knowledge  Guidance  QS_Adoption_Score Performance_Level  \
0       100.0       95.0     100.0               95.0

Clean the data 

In [18]:
#Handle Missing Evaluation Scores for days with no evaluations

#Define the evaluation related colums
eval_cols=["QS_Adoption_Score", "Professionalism", "Logging", "Compliance", "Knowledge", "Efficiency", "Exceptions", "Guidance"]

#Replace NaN with "No Evaluation"
advisorskpi[eval_cols]=advisorskpi[eval_cols].fillna("No Evaluation")

#Remove the duplicate 
advisorskpi_no_duplicates=advisorskpi.drop_duplicates()

#convert numeric columns to proper data 
numeric_cols=[ "AHT", "Schedule_Adherence", "ACW","RONA","Efficiency", "Compliance", "Logging", "Professionalism", "Exceptions", "Knowledge", "Guidance","QS_Adoption_Score"]

# Convert numeric columns to numeric type
advisorskpi[numeric_cols]=advisorskpi[numeric_cols].apply(pd.to_numeric, errors="coerce")

#identify categorial columns 
categorical_cols = advisorskpi.select_dtypes(include='object').columns

print("Categorical columns:", categorical_cols)

#print Data types 
print(advisorskpi.dtypes)

Categorical columns: Index(['Week', 'Employee_ID', 'Performance_Level', 'Needs_Coaching'], dtype='object')
Week                     object
Employee_ID              object
AHT                     float64
Schedule_Adherence      float64
ACW                     float64
RONA                      int64
Evaluation_Completed       bool
Efficiency              float64
Compliance              float64
Logging                 float64
Professionalism         float64
Exceptions              float64
Knowledge               float64
Guidance                float64
QS_Adoption_Score       float64
Performance_Level        object
Needs_Coaching           object
dtype: object


Train a regression Model 

In [21]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Save cleaned dataset to a CSV file 
advisorskpi.to_csv("cleaned_advisorskpi.csv", index=False)

# Load the cleaned CSV file
cleaned_advisorskpi = pd.read_csv("cleaned_advisorskpi.csv")

# Create the next QS score column
cleaned_advisorskpi["Next_QS_Score"] = cleaned_advisorskpi["QS_Adoption_Score"].shift(-1)

# Drop the last row (it has no next QS score)
cleaned_advisorskpi = cleaned_advisorskpi.dropna(subset=["Next_QS_Score"])

# Optional: reset index
cleaned_advisorskpi = cleaned_advisorskpi.reset_index(drop=True)

# Select input features and target variable
features = ["AHT", "ACW", "Schedule_Adherence", "RONA"]
X = cleaned_advisorskpi[features]
y = cleaned_advisorskpi["Next_QS_Score"]

# Scale the input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Train model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Predict on full dataset
cleaned_advisorskpi["Predicted_Next_QS"] = model.predict(X_scaled)



Evaluate the model 

In [23]:
print("MSE:", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))

SyntaxError: invalid non-printable character U+00A0 (4292645000.py, line 2)

Apply the function

In [ ]:
cleaned_advisorskpi["Coaching_Reason"] = cleaned_advisorskpi.apply(coaching_reason, axis=1)

Turning the function into a readable coaching message 

In [ ]:
cleaned_advisorskpi["Coaching_Message"] = cleaned_advisorskpi.apply(
    lambda row: f"Predicted QS: {row['Predicted_Next_QS']:.1f}. Coaching: {row['Coaching_Recommended']}. Reason: {row['Coaching_Reason']}",
    axis=1
)

Define threshold for low predicted score (assuming QS Adoptions core ranges from o to 100, we decide below 80 needs coaching)

In [ ]:
threshold = 80.0

#Mark Yes if predicted next QS is below thhreshold otherwise No 
cleaned_advisorskpi["Coaching_Recommended"] = np.where(cleaned_advisorskpi["Predicted_Next_QS"] < threshold, "Yes", "No")



Explain why the coaching is recommended

In [ ]:
# Explain why coaching is recommended
def coaching_reason(row):
    if row["Predicted_Next_QS"] < 80:
        if row["RONA"] > 1:
            return "RONA too high"
        elif row["AHT"] > 2:
            return "AHT above threshold"
        else:
            return "Low predicted QS score"
    else:
        return "On Track"

Filter the dataframe to show only the relevant columns

In [ ]:
report_cleaned_advisorskpi = cleaned_advisorskpi[["Date", "AHT", "ACW", "RONA", "Schedule_Adherence", 
                "QS_Adoption_Score", "Predicted_Next_QS", 
                "Coaching_Recommended", "Coaching_Reason"]]

Display the coaching report in the streamlit app

In [ ]:
if uploaded_file is not None 

    st.header("Coaching Report")
    # Compute predictions and add report columns as shown above
   cleaned_advisorskpi["Predicted_Next_QS"] = model.predict(cleaned_advisorskpi[features])
    cleaned_advisorskpi["Coaching_Recommended"] = np.where(cleaned_advisorskpi["Predicted_Next_QS"] < threshold, "Yes", "No")
    cleaned_advisorskpi["Coaching_Reason"] = np.where(cleaned_advisorskpi["Coaching_Recommended"] == "Yes",
                                     "Predicted QS adoption score below acceptable level", "")
    report_cleaned_advisorskpi = cleaned_advisorskpi[["Date", "AHT", "ACW", "RONA", "Schedule_Adherence", 
                    "QS_Adoption_Score", "Predicted_Next_QS", 
                    "Coaching_Recommended", "Coaching_Reason"]]

    st.dataframe(report_cleaned_advisorskpi)  # display the report as an interactive table

Add a download coaching report button

In [ ]:
#convert dataframe to csv
csv_data = report_cleaned_advisorskpi.to_csv(index=False).encode('utf-8')

#Add the download button widget
st.download_button(
    label="Download Coaching Report",
    data=csv_data,
    file_name="coaching_report.csv",
    mime="text/csv"
)